# How to Finetune a VLM on Bedrock and deploy on Fireworks

This guide will use Qwen 2.5 VL 7B, but you can use this guide with any other model.

### Pre-requisites
- [aws-cli](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)
- Python

In [1]:
%pip install boto3 "huggingface-hub[cli]"

  Using cached boto3-1.38.14-py3-none-any.whl.metadata (6.6 kB)
  Using cached huggingface_hub-0.31.1-py3-none-any.whl.metadata (13 kB)
  Using cached botocore-1.38.14-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.12.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached InquirerPy-0.3.4-py3-none-any.whl.metadata (8.1 kB)
  Using cached pfzy-0.3.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.

In [4]:
import boto3

account_id = boto3.client('sts').get_caller_identity()['Account']

BUCKET_NAME=f"fw-bedrock-finetuning-{account_id}"
HF_MODEL_NAME="Qwen/Qwen2.5-VL-7B-Instruct"
MODEL_NAME=HF_MODEL_NAME.lower().replace(".", "p").split("/")[-1]

print("BUCKET_NAME", BUCKET_NAME)
print("HF_MODEL_NAME", HF_MODEL_NAME)
print("MODEL_NAME", MODEL_NAME)

BUCKET_NAME fw-bedrock-finetuning-023979239971
HF_MODEL_NAME Qwen/Qwen2.5-VL-7B-Instruct
MODEL_NAME qwen2p5-vl-7b-instruct


In [5]:
s3 = boto3.client('s3')

try:
    s3.create_bucket(Bucket=BUCKET_NAME)
    print(f"Created bucket: {BUCKET_NAME}")
except s3.exceptions.BucketAlreadyExists:
    print(f"Bucket {BUCKET_NAME} already exists")
except s3.exceptions.BucketAlreadyOwnedByYou:
    print(f"Bucket {BUCKET_NAME} already owned by you")

Created bucket: fw-bedrock-finetuning-023979239971


In [6]:
# Download model weights
!huggingface-cli download {HF_MODEL_NAME} --local-dir {MODEL_NAME}

Fetching 16 files:   0%|                                 | 0/16 [00:00<?, ?it/s]Downloading 'model-00001-of-00005.safetensors' to 'qwen2p5-vl-7b-instruct/.cache/huggingface/download/ucOYR0sM0bukMpWiwV3qkLpQbMA=.e97b877e47fde53a6c6e77aafb36e58e91ee9d95c4a3eeac6f1b5c0e6a1c986e.incomplete'
Xet Storage is enabled for this repo. Downloading file from Xet Storage..
Xet Storage is enabled for this repo. Downloading file from Xet Storage..

model-00001-of-00005.safetensors:   0%|             | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|             | 0.00/3.86G [00:00<?, ?B/s]


generation_config.json: 100%|██████████████████| 216/216 [00:00<00:00, 6.34MB/s]
Download complete. Moving file to qwen2p5-vl-7b-instruct/generation_config.json



config.json: 100%|█████████████████████████| 1.37k/1.37k [00:00<00:00, 35.4MB/s]
Download complete. Moving file to qwen2p5-vl-7b-instruct/config.json



chat_template.json: 100%|██████████████████| 1.05k/1.05k [00:00<00:00, 30.0MB/s]


In [ ]:
# Upload model weights to S3 bucket
!aws s3 cp {MODEL_NAME} s3://{BUCKET_NAME}/base/{MODEL_NAME}/ --recursive

upload: qwen2p5-vl-7b-instruct/.cache/huggingface/download/chat_template.json.lock to s3://fw-bedrock-finetuning-023979239971/base/qwen2p5-vl-7b-instruct/.cache/huggingface/download/chat_template.json.lock
upload: qwen2p5-vl-7b-instruct/.cache/huggingface/.gitignore to s3://fw-bedrock-finetuning-023979239971/base/qwen2p5-vl-7b-instruct/.cache/huggingface/.gitignore
upload: qwen2p5-vl-7b-instruct/.cache/huggingface/download/README.md.lock to s3://fw-bedrock-finetuning-023979239971/base/qwen2p5-vl-7b-instruct/.cache/huggingface/download/README.md.lock
upload: qwen2p5-vl-7b-instruct/.cache/huggingface/download/.gitattributes.lock to s3://fw-bedrock-finetuning-023979239971/base/qwen2p5-vl-7b-instruct/.cache/huggingface/download/.gitattributes.lock
upload: qwen2p5-vl-7b-instruct/.cache/huggingface/download/config.json.metadata to s3://fw-bedrock-finetuning-023979239971/base/qwen2p5-vl-7b-instruct/.cache/huggingface/download/config.json.metadata
upload: qwen2p5-vl-7b-instruct/.cache/huggingf

In [ ]:
bedrock = boto3.client("bedrock")

# Create role for job to read S3


response = bedrock.create_model_import_job(
    jobName=MODEL_NAME,
    importedModelName=MODEL_NAME,
    roleArn=,
    modelDataSource={
        "s3DataSource": {
            "s3Uri": f"s3://{BUCKET_NAME}/base/{MODEL_NAME}"
        }
    }
)